# Capstone Project Working Notebook: David

In [1]:
# regular imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import math

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

import warnings
warnings.filterwarnings("ignore")

# Wrangling
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.cluster import KMeans
from scipy import stats
import sklearn.preprocessing
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr, spearmanr, kruskal

import csv
import ds_acquire
import ds_prepare


In [2]:
df = ds_acquire.get_bach_df()
df = ds_prepare.clean_col_names(df)
df = ds_prepare.clean_bach_df(df)

dataframe shape: (71901, 116)
modified df shape: (71901, 101)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71901 entries, 0 to 71900
Columns: 101 entries, Unnamed: 0 to non_deg_seeking
dtypes: float64(79), int64(3), object(19)
memory usage: 55.4+ MB


In [4]:
df.major_name.value_counts()

Business Administration, Management and Operations.           1870
Psychology, General.                                          1565
Biology, General.                                             1468
Liberal Arts and Sciences, General Studies and Humanities.    1434
English Language and Literature, General.                     1409
                                                              ... 
Construction Trades, Other.                                      1
Podiatric Medicine/Podiatry.                                     1
Physics and Astronomy.                                           1
Advanced/Graduate Dentistry and Oral Sciences.                   1
Social Psychology.                                               1
Name: major_name, Length: 375, dtype: int64

In [5]:
df.state_post_code.value_counts()

NY    4829
PA    4709
CA    4195
OH    3616
TX    3456
IL    2524
MA    2346
FL    2253
MI    2233
IN    2068
NC    1986
MO    1907
GA    1671
TN    1644
WI    1630
VA    1622
MN    1616
IA    1387
NJ    1202
SC    1177
WA    1164
CT    1152
KS    1099
KY    1072
OK    1071
AL    1057
PR     963
MD     961
OR     875
LA     859
CO     858
AR     814
NE     775
AZ     733
UT     702
WV     644
MS     584
ME     556
VT     537
NH     509
SD     486
ID     466
RI     435
DC     417
ND     398
NM     374
MT     335
HI     262
NV     249
DE     236
AK     153
WY      71
GU      37
VI      28
MP       6
AS       1
FM       1
MH       1
Name: state_post_code, dtype: int64

In [6]:
df.shape

(71901, 101)

In [7]:
df.head()

,Unnamed: 0,unit_id_institution,college_name,state_post_code,zip_code,city,region_ipeds,title_IV_eligibility,pred_degree,pred_degree_0and4,...,deg_percent_awarded_social_sciences,deg_percent_awarded_construction_trades,deg_percent_awarded_mechanic_repair,deg_percent_awarded_precision_production,deg_percent_awarded_transportation_materials,deg_percent_awarded_visual_and_performing_arts,deg_percent_awarded_health,deg_percent_awarded_business_management,deg_percent_awarded_history,non_deg_seeking
0,0,"100,654.00",Alabama A & M University,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.04,0.00,0.00,0.00,0.00,0.02,0.00,0.16,0.00,153.00
1,1,"100,654.00",Alabama A & M University,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.04,0.00,0.00,0.00,0.00,0.02,0.00,0.16,0.00,153.00
2,2,"100,654.00",Alabama A & M University,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.04,0.00,0.00,0.00,0.00,0.02,0.00,0.16,0.00,153.00
3,5,"100,654.00",Alabama A & M University,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.04,0.00,0.00,0.00,0.00,0.02,0.00,0.16,0.00,153.00
4,6,"100,654.00",Alabama A & M University,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.04,0.00,0.00,0.00,0.00,0.02,0.00,0.16,0.00,153.00


In [9]:
# Forming main df into simplified df with new col names
new_df = ds_prepare.clean_col_names(df)

In [10]:
new_df.shape

(71901, 101)

In [11]:
new_df.head()

,Unnamed: 0,unit_id_institution,college_name,state_post_code,zip_code,city,region_ipeds,title_IV_eligibility,pred_degree,pred_degree_0and4,...,deg_percent_awarded_social_sciences,deg_percent_awarded_construction_trades,deg_percent_awarded_mechanic_repair,deg_percent_awarded_precision_production,deg_percent_awarded_transportation_materials,deg_percent_awarded_visual_and_performing_arts,deg_percent_awarded_health,deg_percent_awarded_business_management,deg_percent_awarded_history,non_deg_seeking
0,0,"100,654.00",Alabama A & M University,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.04,0.00,0.00,0.00,0.00,0.02,0.00,0.16,0.00,153.00
1,1,"100,654.00",Alabama A & M University,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.04,0.00,0.00,0.00,0.00,0.02,0.00,0.16,0.00,153.00
2,2,"100,654.00",Alabama A & M University,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.04,0.00,0.00,0.00,0.00,0.02,0.00,0.16,0.00,153.00
3,5,"100,654.00",Alabama A & M University,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.04,0.00,0.00,0.00,0.00,0.02,0.00,0.16,0.00,153.00
4,6,"100,654.00",Alabama A & M University,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.04,0.00,0.00,0.00,0.00,0.02,0.00,0.16,0.00,153.00


In [12]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71901 entries, 0 to 71900
Columns: 101 entries, Unnamed: 0 to non_deg_seeking
dtypes: float64(79), int64(3), object(19)
memory usage: 55.4+ MB


In [13]:
new_df.major_name.value_counts()

Business Administration, Management and Operations.           1870
Psychology, General.                                          1565
Biology, General.                                             1468
Liberal Arts and Sciences, General Studies and Humanities.    1434
English Language and Literature, General.                     1409
                                                              ... 
Construction Trades, Other.                                      1
Podiatric Medicine/Podiatry.                                     1
Physics and Astronomy.                                           1
Advanced/Graduate Dentistry and Oral Sciences.                   1
Social Psychology.                                               1
Name: major_name, Length: 375, dtype: int64

In [20]:
major_list = new_df.major_name.unique().tolist()

In [21]:
major_list

['Agriculture, General.',
 'Animal Sciences.',
 'Food Science and Technology.',
 'Plant Sciences.',
 'Agriculture, Agriculture Operations, and Related Sciences, Other.',
 'Forestry.',
 'City/Urban, Community and Regional Planning.',
 'Audiovisual Communications Technologies/Technicians.',
 'Computer and Information Sciences, General.',
 'Special Education and Teaching.',
 'Teacher Education and Professional Development, Specific Levels and Methods.',
 'Teacher Education and Professional Development, Specific Subject Areas.',
 'Civil Engineering.',
 'Electrical, Electronics and Communications Engineering.',
 'Mechanical Engineering.',
 'Electrical Engineering Technologies/Technicians.',
 'Mechanical Engineering Related Technologies/Technicians.',
 'Construction Engineering Technologies.',
 'Family and Consumer Sciences/Human Sciences, General.',
 'English Language and Literature, General.',
 'Liberal Arts and Sciences, General Studies and Humanities.',
 'Biology, General.',
 'Mathematic

In [24]:
new_df.major_name.value_counts()

Business Administration, Management and Operations.           1870
Psychology, General.                                          1565
Biology, General.                                             1468
Liberal Arts and Sciences, General Studies and Humanities.    1434
English Language and Literature, General.                     1409
                                                              ... 
Construction Trades, Other.                                      1
Podiatric Medicine/Podiatry.                                     1
Physics and Astronomy.                                           1
Advanced/Graduate Dentistry and Oral Sciences.                   1
Social Psychology.                                               1
Name: major_name, Length: 375, dtype: int64

data = [major_list]

file = open('major_list.csv', 'w+', newline='')

with file:
    write = csv.writer(file)
    write.writerows(data)


In [ ]:
new_df['major_category'] = new_df.major_name({
    'Agriculture': ['Botany/Plant Biology.','Agricultural Engineering.','Applied Horticulture and Horticultural Business Services.','Agriculture/Veterinary Preparatory Programs.','Soil Sciences.','Agriculture, General.', 'Agriculture, Agriculture Operations, and Related Sciences, Other.', 'Agricultural Production Operations.', 'Agricultural and Domestic Animal Services.','Agricultural Public Services.','Agricultural Mechanization.','International Agriculture.','Agricultural and Food Products Processing.'],
    'Environment and Natural Resources': ['Forest Engineering.','Environmental Control Technologies/Technicians.','Archeology.','Geological/Geophysical Engineering.','Mining and Mineral Engineering.','Natural Resources and Conservation, Other.','Fishing and Fisheries Sciences and Management.','Sustainability Studies.','Historic Preservation and Conservation.','Surveying Engineering.','Ocean Engineering.','Geography and Cartography.','Wildlife and Wildlands Science and Management.','Natural Resources Management and Policy.','Geological and Earth Sciences/Geosciences.','Environmental/Environmental Health Engineering.','Natural Resources Conservation and Research.','Forestry.','Parks, Recreation and Leisure Facilities Management.'],
    'Architecture': ['Drafting/Design Engineering Technologies/Technicians.','Architecture and Related Services, Other.','Architectural Sciences and Technology.','Interior Architecture.','Architectural Engineering.','Architecture.','Environmental Design.'],
    'Area, Ethnic, and Civilization Studies': ['Demography and Population Studies.','Medieval and Renaissance Studies.','Classical and Ancient Studies.','Bilingual, Multilingual, and Multicultural Education.','Museology/Museum Studies.','Science, Technology and Society.','Urban Studies/Affairs.','Cultural Studies/Critical Theory and Analysis.','African Languages, Literatures, and Linguistics.', 'Turkic, Uralic-Altaic, Caucasian, and Central Asian Languages, Literatures, and Linguistics.'],
    'Communications': ['Telecommunications Management.','Communication, Journalism, and Related Programs, Other.','Communication and Media Studies.','Public Relations, Advertising, and Applied Communication.'],
    'Communication Technologies': ['Communications Technologies/Technicians and Support Services, Other.','Educational/Instructional Media Design.','Graphic Communications.','Communications Technology/Technician.','Audiovisual Communications Technologies/Technicians.','Radio, Television, and Digital Communication.'],
    'Computer and Information Sciences': ['Accounting and Computer Science.','Human Computer Interaction.','Data Processing.','Computational Science.','Computer Software and Media Applications.','Computer and Information Sciences and Support Services, Other.','Computer Engineering Technologies/Technicians.','Computer Systems Analysis.','Computer Systems Networking and Telecommunications.','Computer Programming.','Computer/Information Technology Administration and Management.','Computer Science.','Information Science/Studies.','Computer Engineering.','Computer and Information Sciences, General.','Management Information Systems and Services.'],
    'Cosmetology Services and Culinary Arts': ['Personal and Culinary Services, Other.','Cosmetology and Related Personal Grooming Services.','Nutrition Sciences.','Culinary Arts and Related Services.'],
    'Education Administration and Teaching': ['Basic Skills and Developmental/Remedial Education.','Curriculum and Instruction.','High School/Secondary Diploma Programs.','High School/Secondary Certificate Programs.','Social and Philosophical Foundations of Education.','Teaching Assistants/Aides.','Student Counseling and Personnel Services.','Educational Administration and Supervision.','Teaching English or French as a Second or Foreign Language.','Education, Other.','Educational Assessment, Evaluation, and Research.','Education, General.','Special Education and Teaching.','Teacher Education and Professional Development, Specific Levels and Methods.','Teacher Education and Professional Development, Specific Subject Areas.'],
    'Engineering': ['Ceramic Sciences and Engineering.','Electromechanical Engineering.','Biochemical Engineering.','Engineering Chemistry.','Mechatronics, Robotics, and Automation Engineering.','Engineering Mechanics.','Engineering Physics.','Engineering-Related Fields.','Engineering Science.','Petroleum Engineering.','Metallurgical Engineering.','Engineering, Other.','Industrial Engineering.','Chemical Engineering.','Aerospace, Aeronautical and Astronautical Engineering.','Engineering, General.','Electrical, Electronics and Communications Engineering.','Civil Engineering.','Mechanical Engineering.','Materials Engineering'],
    'Engineering Technologies': ['Civil Engineering Technologies/Technicians.','Engineering-Related Technologies.','Engineering Technologies/Technicians, Other.','Engineering Technology, General.','Electrical Engineering Technologies/Technicians.','Mechanical Engineering Related Technologies/Technicians.','Construction Engineering Technologies.'],
    'Linguistics and Foreign Languages': ['Iranian/Persian Languages, Literatures, and Linguistics.','Turkic, Uralic-Altaic, Caucasian, and Central Asian Languages, Literatures, and Linguistics.','African Languages, Literatures, and Linguistics.','Celtic Languages, Literatures, and Linguistics.','South Asian Languages, Literatures, and Linguistics.','Middle/Near Eastern and Semitic Languages, Literatures, and Linguistics.','American Sign Language.','Slavic, Baltic and Albanian Languages, Literatures, and Linguistics.','Foreign Languages, Literatures, and Linguistics, Other.','American Indian/Native American Languages, Literatures, and Linguistics.','East Asian Languages, Literatures, and Linguistics.','Germanic Languages, Literatures, and Linguistics.','Modern Greek Language and Literature.','Southeast Asian and Australasian/Pacific Languages, Literatures, and Linguistics.','Linguistic, Comparative, and Related Language Studies and Services.','Romance Languages, Literatures, and Linguistics.'],
    'Family and Consumer Sciences': ['Work and Family Studies.','Family and Consumer Sciences/Human Sciences Business Services.','Family and Consumer Sciences/Human Sciences, General.','Family and Consumer Sciences/Human Sciences, Other.','Hospitality Administration/Management.','Family and Consumer Economics and Related Studies.'],
    'Law': ['Law.','Legal Professions and Studies, Other.','Legal Research and Advanced Professional Studies.','Legal Support Services.','Non-Professional General Legal Studies (Undergraduate).'],
    'English Language, Literature, and Composition': ['Creative Writing.','Publishing.','English Language and Literature/Letters, Other.','Literature.','Classics and Classical Languages, Literatures, and Linguistics.','English Language and Literature, General.','Journalism.'],
    'Liberal Arts and Humanities': ['Liberal Arts and Sciences, General Studies and Humanities.'],
    'Library Science': ['Library Science and Administration.','Library Science, Other.'],
    'Biology and Life Sciences': ['Neuroscience.','Nanotechnology.','Biology Technician/Biotechnology Laboratory Technician.','Veterinary Medicine.','Maritime Studies.','Marine Sciences.','Pharmacology and Toxicology.','Human Biology.','Veterinary Biomedical and Clinical Sciences.','Atmospheric Sciences and Meteorology.','Biomathematics, Bioinformatics, and Computational Biology.','Cell/Cellular Biology and Anatomical Sciences.','Biological and Physical Sciences.','Biochemistry, Biophysics and Molecular Biology.','Zoology/Animal Biology.','Veterinary/Animal Health Technologies/Technicians.','Microbiological Sciences and Immunology.','Foods, Nutrition, and Related Services.','Ecology, Evolution, Systematics, and Population Biology.','Neurobiology and Neurosciences.','Genetics.','Animal Sciences.','Plant Sciences.','Food Science and Technology.','Chemistry.','Biology, General.','Biomedical/Medical Engineering.'],
    'Mathematics and Statistics': ['Mathematics and Statistics, Other.','Mathematics and Computer Science.','Physics and Astronomy.','Statistics.','Mathematics.','Physics.','Astronomy and Astrophysics.','Applied Mathematics.'],
    'Military Technologies': ['Military Science and Operational Studies.','Military Technologies and Applied Sciences, Other.','Air Force ROTC, Air Science and Operations.','Army ROTC, Military Science and Operations.','Intelligence, Command Control and Information Operations.','Naval Architecture and Marine Engineering.','Military Systems and Maintenance Technology.','Military Applied Sciences.','Security Science and Technology.'],
    'Interdisciplinary and Multi-Disciplinary Studies (General)': ['International and Comparative Education.','Systems Science and Theory.','Intercultural/Multicultural and Diversity Studies.','International/Global Studies.','Multi-/Interdisciplinary Studies, General.','Multi/Interdisciplinary Studies, Other.','Area Studies.'],
    'Physical Fitness, Parks, Recreation, and Leisure': ['Somatic Bodywork and Related Therapeutic Services.','Movement and Mind-Body Therapies and Education.','Leisure and Recreational Activities.','Housing and Human Environments.','Landscape Architecture.','Outdoor Education.','Parks, Recreation, Leisure, and Fitness Studies, Other.','Health and Physical Education/Fitness.','Parks, Recreation and Leisure Studies.'],
    'Philosophy and Religious Studies': ['Philosophy and Religious Studies, Other.','Philosophy and Religious Studies, General.','Religious Education.','Philosophy.','Bioethics/Medical Ethics.','Religious/Sacred Music.'],
    'Theology and Religious Vocations': ['Theology and Religious Vocations, Other.','Theological and Ministerial Studies.','Missions/Missionary Studies and Missiology.','Religion/Religious Studies.','Bible/Biblical Studies.','Pastoral Counseling and Specialized Ministries.'],
    'Physical Sciences': ['Energy and Biologically Based Therapies.','Physical Science Technologies/Technicians.','Physiology, Pathology and Related Sciences.','Natural Sciences.','Physical Sciences.','Physical Sciences, Other.'],
    'Nuclear, Industrial Radiology, and Biological Technologies': ['Nuclear and Industrial Radiologic Technologies/Technicians.','Nuclear Engineering.','Nuclear Engineering Technologies/Technicians.','Science Technologies/Technicians, Other.','Electromechanical Instrumentation and Maintenance Technologies/Technicians.'],
    'Psychology': ['Social Psychology.','Interpersonal and Social Skills.','Cognitive Science.','Biopsychology.','Research and Experimental Psychology.','Psychology, Other.','Clinical, Counseling and Applied Psychology.','Behavioral Sciences.','Clinical Psychology.','Human Development, Family Studies, and Related Services.'],
    'Criminal Justice and Fire Protection': ['Homeland Security.','Homeland Security, Law Enforcement, Firefighting and Related Protective Services, Other.','International Relations and National Security Studies.','Fire Protection.','Criminal Justice and Corrections.','Criminology.'],
    'Public Affairs, Policy, and Social Work': ['Security Policy and Strategy.','Taxation.','Citizenship Activities.','Peace Studies and Conflict Resolution.','Human Services, General.','Community Organization and Advocacy.','Mental and Social Health Services and Allied Professions.','Public Policy Analysis.','Public Administration and Social Service Professions, Other.','Public Administration.','Economics.','Rehabilitation and Therapeutic Professions.','City/Urban, Community and Regional Planning.','Social Work.','Political Science and Government.'],
    'Social Sciences': ['Dispute Resolution.','Sociology and Anthropology.','Rural Sociology.','Social Sciences, General.','Communication Disorders Sciences and Services.','Human Development, Family Studies, and Related Services.','Sociology.','Psychology, General.','Ethnic, Cultural Minority, Gender, and Group Studies.','Anthropology.','Social Sciences, Other.'],
    'Construction Services': ['Carpenters.','Mason/Masonry.','Construction Trades, Other.','Construction Trades, General.','Woodworking.','Electrical and Power Transmission Installers.','Construction Management.','Building/Construction Finishing, Management, and Inspection.','Architectural Engineering Technologies/Technicians.','Heating, Air Conditioning, Ventilation and Refrigeration Maintenance Technology/Technician (HAC, HACR, HVAC, HVACR).','Construction Engineering.'],
    'Electrical and Mechanic Repairs and Technologies': ['Heavy/Industrial Equipment Maintenance Technologies.','Vehicle Maintenance and Repair Technologies.','Electrical/Electronics Maintenance and Repair Technology.','Science Technologies/Technicians, General.','Energy Systems Technologies/Technicians.'],
    'Precision Production and Industrial Arts': ['Paper Science and Engineering.','Precision Metal Working.','Materials Sciences.','Systems Engineering.','Manufacturing Engineering.','Quality Control and Safety Technologies/Technicians.','Industrial Production Technologies/Technicians.','Polymer/Plastics Engineering.','Apparel and Textiles.','Textile Sciences and Engineering.'],
    'Transportation Sciences and Technologies': ['Mining and Petroleum Technologies/Technicians.','Marine Transportation.','Air Transportation.','Transportation and Materials Moving, Other.'],
    'Fine Arts': ['Crafts/Craft Design, Folk Art and Artisanry.','Visual and Performing Arts, Other.','Film/Video and Photographic Arts.','Visual and Performing Arts, General.','Design and Applied Arts.','Dance.','Rhetoric and Composition/Writing Studies.','Fine and Studio Arts.','Music.','Drama/Theatre Arts and Stagecraft.'],
    'Medical and Health Sciences and Services': ['Medical Clinical Sciences/Graduate Medical Studies.','Dentistry.','Alternative and Complementary Medical Support Services.','Optometry.','Health-Related Knowledge and Skills.','Funeral Service and Mortuary Science.','Gerontology.','Ophthalmic and Optometric Support Services and Allied Professions.','Alternative and Complementary Medicine and Medical Systems.','Chiropractic.','Podiatric Medicine/Podiatry.','Advanced/Graduate Dentistry and Oral Sciences.','Alternative and Complementary Medicine and Medical Systems.''Biological and Biomedical Sciences, Other.','Practical Nursing, Vocational Nursing and Nursing Assistants.','Pharmacy, Pharmaceutical Sciences, and Administration.','Medicine.','Medical Illustration and Informatics.','Allied Health and Medical Assisting Services.','Dental Support Services and Allied Professions.','Health/Medical Preparatory Programs.','Biological/Biosystems Engineering.','Biotechnology.','Nursing.','Health Professions and Related Clinical Sciences, Other.','Dietetics and Clinical Nutrition Services.','Registered Nursing, Nursing Administration, Nursing Research and Clinical Nursing.','Clinical/Medical Laboratory Science/Research and Allied Professions.','Public Health.','Health Services/Allied Health/Health Sciences, General.','Health and Medical Administrative Services.','Allied Health Diagnostic, Intervention, and Treatment Professions.'],
    'Business': ['Real Estate Development.','Operations Research.','Real Estate.','Insurance.','Specialized Sales, Merchandising and  Marketing Operations.','Arts, Entertainment,and Media Management.','Business Operations Support and Assistant Services.','Management Sciences and Quantitative Methods.','Business, Management, Marketing, and Related Support Services, Other.','Business/Commerce, General.','International Business.','Agricultural Business and Management.','Human Resources Management and Services.','General Sales, Merchandising and Related Marketing Operations.','Business/Managerial Economics.','Business/Corporate Communications.','Business Administration, Management and Operations.','Accounting and Related Services.','Entrepreneurial and Small Business Operations.','Finance and Financial Management Services.','Marketing.'],
    'History': ['History.','Holocaust and Related Studies.','Architectural History and Criticism.'],
})